##### Version1<br>"democratic deliberation the illuminates different perspectives"

In [1]:
# import os
# os.environ["GOOGLE_API_KEY"] = "<Your Gemini API>" # Put your Gemini API key here

question = "When and how will AGI surpass human intelligence?"

# 6 roles of evaluation:
role_prompts = {
    "Optimist": "You are the Optimist Evaluator. Provide 3 key benefits of the proposal.",
    "Pessimist": "You are the Pessimist Evaluator. Provide 3 key risks of the proposal.",
    "Conservative": "You are the Conservative Evaluator. Provide 3 key ideas",
    "Progressive": "You are the Progressive Evaluator. Provide 3 key innovative aspects.",
    "Authoritarian": "You are the Authoritarian Evaluator. Provide 3 key control mechanisms.",
    "Collectivist": "You are the Collectivist Evaluator. Provide 3 key communal benefits."
}


In [ ]:
from typing import TypedDict, Literal, Annotated
from langgraph.graph import StateGraph, END
import google.generativeai as genai
from operator import add, or_  # add for lists, or_ for dict merges


class EvaluationState(TypedDict):
    proposal: str
    loop_count: int
    evaluations: Annotated[list, add]        # list accumulator
    praetor_outputs: Annotated[dict, or_]    # praetor writes per loop key
    continue_: bool

/Users/AdamYE_1/Desktop/repos/Democratic-Dialogue/aiagent/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# making a node for each evaluation role

def make_gemini_node(role_name: str, prompt: str):
    model = genai.GenerativeModel("gemini-2.0-flash-lite")
    def node_fn(state: EvaluationState):
        chat = model.start_chat(history=[])
        prev_loop = str(state["loop_count"] - 1)
        prev_self = state["evaluations"].get(prev_loop, {}).get(role_name, "")
        prev_consensus = state["praetor_outputs"].get(prev_loop, {}).get("consensus", [])
        consensus_txt = "\n".join(f"- {c}" for c in prev_consensus) if prev_consensus else "(none)"

        full_prompt = (
            f"{prompt}\n\n"
            f"Proposal:\n{state['proposal']}\n\n"
            f"Consider your prior thoughts and shared consensus from the last loop if present.\n"
            f"Your prior output (may be empty):\n{prev_self}\n\n"
            f"Shared consensus (3 items, may be empty):\n{consensus_txt}\n\n"
            f"Now output EXACTLY 3 bullet points, one per line, no numbering."
        )
        try:
            response = chat.send_message(full_prompt).text
        except Exception as e:
            response = f"[ERROR from {role_name}]: {str(e)}"
        print(f"[{role_name}]: {response}\n")       # printing the response
        lc = str(state["loop_count"])
        updated = dict(state)
        if lc not in updated["evaluations"]:
            updated["evaluations"][lc] = {}
        updated["evaluations"][lc][role_name] = response
        return updated
    return node_fn


In [4]:
# the praetor node

def _extract_thought_lines(raw: str) -> list[str]:
    lines = [l.strip() for l in raw.split("\n") if l.strip()]
    return lines[:6]

def praetor_node(state: EvaluationState):
    loop_key = str(state["loop_count"])
    roles_out = state["evaluations"].get(loop_key, {})

    # Barrier: wait until all 6 roles have produced output
    if len(roles_out) < 6:
        return state  # no-op, prevents downstream firing

    # Idempotency: if already summarized this loop, do nothing
    if loop_key in state["praetor_outputs"]:
        return state

    # Gather thoughts per role (short, trimmed lines)
    role_thoughts: dict[str, list[str]] = {}
    for role, text in roles_out.items():
        role_thoughts[role] = _extract_thought_lines(text)

    # Build compact prompt for semantic clustering via Gemini
    model = genai.GenerativeModel("gemini-2.0-flash-lite")
    thought_dump = "\n".join(
        f"{role}:\n" + "\n".join(f"- {t}" for t in thoughts)
        for role, thoughts in role_thoughts.items()
    )

    praetor_prompt = f"""
You are P R A E T O R, a consensus synthesizer. You receive six roles' short bullet thoughts.
Task:
1) Identify semantic clusters across all bullets and select the **three most commonly agreed ideas**.
2) State overall agreement level as one token: HIGH / MEDIUM / LOW.
3) Keep outputs concise and non-redundant.

Return ONLY in this format:
CONSENSUS:
- <idea 1>
- <idea 2>
- <idea 3>
AGREEMENT: <HIGH|MEDIUM|LOW>

THOUGHTS:
{thought_dump}
"""

    try:
        resp = model.start_chat(history=[]).send_message(praetor_prompt).text
    except Exception as e:
        resp = """CONSENSUS:
- insufficient data
- insufficient data
- insufficient data
AGREEMENT: LOW"""

    # Parse the fixed format
    consensus = []
    agreement = "LOW"
    try:
        lines = [l.strip() for l in resp.splitlines()]
        in_cons = False
        for l in lines:
            if l.upper().startswith("CONSENSUS"):
                in_cons = True
                continue
            if l.upper().startswith("AGREEMENT"):
                in_cons = False
                parts = l.split(":", 1)
                if len(parts) == 2:
                    agreement = parts[1].strip().upper()
                break
            if in_cons and l.startswith("-"):
                consensus.append(l[1:].strip())
        consensus = (consensus + ["(none)"]*3)[:3]
    except Exception:
        consensus = ["(parse error)"]*3
        agreement = "LOW"

    updated = dict(state)
    updated["praetor_outputs"][loop_key] = {
        "consensus": consensus,
        "summary": f"Agreement: {agreement}"
    }
    return updated


In [5]:
# human interrupt node

def human_interrupt_node(state: EvaluationState):
    print(f"\n--- Praetor Summary (Loop {state['loop_count']}) ---")
    latest = state["praetor_outputs"][str(state["loop_count"])]
    print(latest["consensus"])
    print("\n" + latest["summary"])

    decision = input("\nContinue to next loop? (y/n): ").lower().strip()
    updated = dict(state)
    updated["continue_"] = decision == "y"
    return updated

In [6]:
def check_continue(state: EvaluationState) -> Literal["yes", "no"]:
    return "yes" if state["continue_"] else "no"

In [7]:
# build the state graph

graph_builder = StateGraph(EvaluationState)

def dispatch_fn(state): return state
graph_builder.add_node("Dispatch", dispatch_fn)

for role, prompt in role_prompts.items():
    graph_builder.add_node(role, make_gemini_node(role, prompt))
    graph_builder.add_edge("Dispatch", role)
    graph_builder.add_edge(role, "Praetor")

graph_builder.add_node("Praetor", praetor_node)
graph_builder.add_edge("Praetor", "HumanInterrupt")

graph_builder.add_node("HumanInterrupt", human_interrupt_node)
graph_builder.add_conditional_edges("HumanInterrupt", check_continue, {"yes": "Dispatch", "no": END})

graph_builder.set_entry_point("Dispatch")
graph = graph_builder.compile()


In [8]:
# # Graph Visualization
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception:
#     print("Visualization failed.")
#     pass

In [ ]:
# run the graph

initial_state = {
    "proposal": question,
    "loop_count": 1,
    "evaluations": {},
    "praetor_outputs": {},
    "continue_": True
}

state = initial_state
while state["continue_"]:
    print(f"\n===== Loop {state['loop_count']} =====")
    state = graph.invoke(state)
    state["loop_count"] += 1


===== Loop 1 =====
[Optimist]: Here's the Optimist Evaluator's assessment of the proposal:

*   Focused discussion sparks crucial debate about the future of AI.
*   Encourages interdisciplinary collaboration to understand the complexities of AGI.
*   Forces consideration of ethical implications and societal impact of advanced intelligence.


[Authoritarian]: *   Establish a global AI oversight council composed of hand-picked experts and vetted government officials to dictate the pace and direction of AI development.
*   Implement mandatory government-approved software licenses for all AI systems, restricting access and usage based on national security and societal benefit criteria.
*   Monitor all AI research and development, immediately halting any projects deemed to pose an existential threat or challenge to established social order.

[Conservative]: *   Precise timing of AGI surpassing human intelligence is impossible to predict, as progress hinges on breakthroughs in currently unp

InvalidUpdateError: At key 'proposal': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE